<a href="https://colab.research.google.com/github/qw-4735/PyTorch/blob/main/%EB%AA%A8%EB%8D%B8_%EB%A7%A4%EA%B0%9C%EB%B3%80%EC%88%98_%EC%B5%9C%EC%A0%81%ED%99%94%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [2]:
# 하이퍼파라미터
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# 손실함수 초기화
loss_fn = nn.CrossEntropyLoss()

In [4]:
# 옵티마이저 초기화
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

학습 단계 (loop)에서 최적화는 세단계로 이루어짐
- optimizer.zero_grad() 를 호출하여 모델 매개변수의 변화도를 재설정.
기본적으로 변화도는 더해지기 때문에, 중복계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정
- loss.backwards() 를 호출하여 prediction loss를 역전파.
파이토치는 각 매개변수에 대한 손실의 변화도를 저장
- 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정

### 전체 구현
- 최적화 코드를 반복하여 수행하는 train_loop와 테스트 데이터로 모델의 성능을 측정하는 test_loop 를 정의

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X,y) in enumerate(dataloader):
    # 예측(prediction)과 손실(loss) 계산
    pred = model(X)
    loss = loss_fn(pred, y)
    # 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X,y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")      

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
   print(f"Epoch {t+1}\n-------------------------------")
   train_loop(train_dataloader, model, loss_fn, optimizer)
   test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306607  [    0/60000]
loss: 2.296937  [ 6400/60000]
loss: 2.284806  [12800/60000]
loss: 2.269077  [19200/60000]
loss: 2.261621  [25600/60000]
loss: 2.225854  [32000/60000]
loss: 2.228217  [38400/60000]
loss: 2.202372  [44800/60000]
loss: 2.195895  [51200/60000]
loss: 2.161409  [57600/60000]
Test Error: 
 Accuracy: 47.4%, Avg loss: 2.162499 

Epoch 2
-------------------------------
loss: 2.179148  [    0/60000]
loss: 2.166824  [ 6400/60000]
loss: 2.119764  [12800/60000]
loss: 2.121229  [19200/60000]
loss: 2.080729  [25600/60000]
loss: 2.027120  [32000/60000]
loss: 2.035790  [38400/60000]
loss: 1.976607  [44800/60000]
loss: 1.974409  [51200/60000]
loss: 1.890752  [57600/60000]
Test Error: 
 Accuracy: 57.2%, Avg loss: 1.899496 

Epoch 3
-------------------------------
loss: 1.941101  [    0/60000]
loss: 1.909580  [ 6400/60000]
loss: 1.800825  [12800/60000]
loss: 1.819631  [19200/60000]
loss: 1.716563  [25600/60000]
loss: 1.681166  [32000/600